In [2]:
%load_ext autoreload
%autoreload 2
import time
import h5py
from itertools import combinations, product
from juliacall import Main as jl
from pathlib import Path
jl.include("/data-local/anlab/Chunxu/DL_MOTIF/4_MOTIF/Unrolled3D/datasets/decolearn.jl")
jl.seval("using Glob")

In [8]:
data_dir = "/data-local/anlab/Chunxu/DL_MOTIF/recon_results/CCIR_01168_ONC-DCE"
raw_data_list = jl.glob("ONC-DCE-*/*.h5",str(data_dir))
start = time.time()

recon_update_list = [ str(Path(folder).parent / "recon_update.h5") for folder in raw_data_list]


In [9]:
with h5py.File(raw_data_list[0], 'r') as f:
    # spoke_len, spokes_num, partition_num, ch_num, phase_num, contrast_num
    t, ph, ch, part, sp, sp_len = f['kspace_data_real'].shape
data = []
for raw_data_path, recon_update_path in zip( raw_data_list, recon_update_list):
    iter_t_ph = product(range(t),combinations(range(ph), 2))
    for t, pair in iter_t_ph:
        item_dict = dict()
        item_dict["source"] = (raw_data_path,t,pair[0])
        item_dict["target"] = (raw_data_path,t,pair[1])
        # item_dict["recon_update_source"] = (recon_update_path,t,pair[0])
        # item_dict["recon_update_source"] = (recon_update_path,t,pair[1])
        data.append(item_dict)

In [21]:
from monai.data import PatchIterd, Dataset, GridPatchDataset
from monai.transforms.adaptors import adaptor
from monai.transforms import RandSpatialCropSamplesd,Transform,MapTransform,Compose,ToTensord,AddChanneld
import numpy as np
import einops as eo
import torch
# def get_data_from_h5(file_path, contrast, phase):
#     with h5py.File(file_path, 'r') as f:
#         d = dict(
#         kspace_data = f['kspace_data_real'][contrast,phase]+1j*f['kspace_data_imag'][contrast,phase],
#         kspace_traj= f['kspace_traj_real'][contrast,phase]+1j*f['kspace_traj_imag'][contrast,phase],
#         kspace_density_compensation = f['kspace_density_compensation'][contrast,phase],
#         cse = f['cse_real'][:]+1j*f['cse_imag'][:],
#         image = f['real'][:]+1j*f['imag'][contrast,phase])
#     return d
class ONC_DCE_H5_Read_Transform(MapTransform):
    def __call__(self, item_dict):
        source_dict = jl.get_data_from_h5(*( item_dict["source"] ))
        target_dict = jl.get_data_from_h5(*( item_dict["target"] ))
        output_dict = dict()
        for k,v in source_dict.items():
            output_dict[k+"_0"] = v.to_numpy()
            print(v.shape)
        for k,v in target_dict.items():
            output_dict[k+"_1"] = v.to_numpy()
        return output_dict
train_transforms_before_patchify = Compose([
    ONC_DCE_H5_Read_Transform(keys=["source","target"]),
    AddChanneld(keys=['image_0','image_1']),
    ToTensord(device= torch.device('cpu'),keys=['kspace_density_compensation_0', 'image_0', 'kspace_data_0', 'kspace_traj_0', 'cse_0', 'kspace_density_compensation_1', 'image_1', 'kspace_data_1', 'kspace_traj_1', 'cse_1']),
    ]) #'recon_update_0','recon_update_1,
n_samples = 16
patch_size = (20, 128, 128)
patch_iter = PatchIterd(keys=['image_0','cse_0','image_1','cse_1']
           ,patch_size=patch_size, start_pos=(0, 0))
# sampler = RandSpatialCropSamplesd(
#     keys= ['recon_update_0','recon_update_1','image_0','cse_0','image_1','cse_1'],
#     roi_size=(128, 128,20), num_samples=16,
#                                  random_center=True, random_size=False)
d = Dataset(data, transform=train_transforms_before_patchify)
d_patchify = GridPatchDataset(d,patch_iter=patch_iter)

# phase_paired_dataset = Dataset(data,
#     transform = )

/data-local/anlab/Chunxu/miniconda3/lib/python3.10/site-packages/monai/utils/deprecate_utils.py:107: FutureWarning: <class 'monai.transforms.utility.array.AddChannel'>: Class `AddChannel` has been deprecated since version 0.8. please use MetaTensor data type and monai.transforms.EnsureChannelFirst instead.
  warn_deprecated(obj, msg, warning_category)


In [ ]:
from monai.data import DataLoader

for item in DataLoader(d_patchify, batch_size=1, num_workers=0):
    print("coordinates:", item[1])
    # break

In [6]:
print(item[0].keys())
item[0]["image_0"].shape

dict_keys(['image_0', 'cse_0', 'image_1', 'cse_1', 'kspace_traj_0', 'kspace_data_1', 'kspace_density_compensation_1', 'kspace_density_compensation_0', 'kspace_traj_1', 'kspace_data_0', 'patch_coords', 'original_spatial_shape', 'start_pos'])


torch.Size([32, 1, 20, 128, 128])

In [1]:
from ONC_DCE_DeCoLearn3D import ONC_DCE_DeCoLearn3D

ds = ONC_DCE_DeCoLearn3D()
ds.prepare_data()
ds.setup()
for item, coord in ds.train_dataloader():
    print(item)
    break

2023-03-28 01:11:55,075 - Created a temporary directory at /tmp/tmp_fnymtbm
2023-03-28 01:11:55,077 - Writing /tmp/tmp_fnymtbm/_remote_module_non_scriptable.py


Gtk-Message: 01:11:59.838: Failed to load module "appmenu-gtk-module"
Gtk-Message: 01:11:59.838: Failed to load module "appmenu-gtk-module"
Gtk-Message: 01:11:59.838: Failed to load module "appmenu-gtk-module"
/data-local/anlab/Chunxu/miniconda3/lib/python3.10/site-packages/monai/utils/deprecate_utils.py:107: FutureWarning: <class 'monai.transforms.utility.array.AddChannel'>: Class `AddChannel` has been deprecated since version 0.8. please use MetaTensor data type and monai.transforms.EnsureChannelFirst instead.
  warn_deprecated(obj, msg, warning_category)


(15, 640)
(80, 320, 320)
(15, 80, 15, 640)
(15, 640)
(15, 80, 320, 320)
[{'image_0': tensor([[[[[-2.9503e-01+9.6413e-03j, -4.2152e-01-8.0762e-02j,
            -2.1965e-01-9.0406e-02j,  ...,
            -3.5790e-01-6.1325e-02j, -1.0033e-01+1.8719e-01j,
            -1.5520e-01+2.2181e-01j],
           [-2.5853e-01+1.3598e-02j, -3.0696e-01-9.6558e-02j,
            -2.7570e-01-1.0246e-01j,  ...,
            -4.1517e-01-9.1012e-02j, -3.3516e-01-3.8008e-03j,
            -1.9393e-02+1.4525e-01j],
           [-5.1300e-01+4.7859e-03j, -3.5523e-01-5.7247e-02j,
            -3.9124e-01+1.1174e-01j,  ...,
            -3.1948e-01-9.2148e-02j, -4.3150e-01-1.4793e-02j,
            -3.2404e-01+3.3642e-01j],
           ...,
           [-4.8448e-01+8.9821e-02j, -4.3466e-01+1.4075e-01j,
            -3.0595e-01-1.2304e-01j,  ...,
            -1.1133e-01+1.1626e-01j, -3.0720e-01+4.2498e-01j,
             1.6361e-01+2.3617e-01j],
           [-9.1099e-02-1.4953e-01j, -5.0246e-01-5.4337e-02j,
            -5.13